# Prediction
This notebook will attempt to build a model to predict the outcome of the passangers

In [10]:
import pandas as pd
import numpy as np

from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.cross_validation import KFold

## Read data 
Column definitions:  
  
| Variable | Definition                                 | Key                                            |
|----------|--------------------------------------------|------------------------------------------------|
| survived | Survived                                   | 0 = No, 1 = Yes                                |
| pclass   | Ticket class                               | 1 = 1st, 2 = 2nd, 3 = 3rd                      |
| sex	   | Sex	                                    | 0 = Female, 1 = Male                           |
| Age	   | Age in years	                            |                                                |
| sibsp	   | # of siblings / spouses aboard the Titanic |	                                             |
| parch	   | # of parents / children aboard the Titanic	|                                                |
| fare	   | Passenger fare	                            |                                                |
| embarked | Port of Embarkation                        | 0 = Southampton, 1 = Cherbourg, 2 = Queenstown |

In [30]:
train = pd.read_csv("./data/clean/train.csv").head(10)
test = pd.read_csv("./data/clean/test.csv").head(10)
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22,1,0,7.2500,0
1,1,1,0,38,1,0,71.2833,1
2,1,3,0,26,0,0,7.9250,0
3,1,1,0,35,1,0,53.1000,0
4,0,3,1,35,0,0,8.0500,0


### Create test datasets
Create Numpy arrays of train, test and target (Survived) dataframes to feed into our models

In [31]:
y_train = train['Survived'].ravel()
train = train.drop(['Survived'], axis=1)
x_train = train.values # Creates an array of the train data
x_test = test.values # Creats an array of the test data

## Helpers

In [32]:
# Some useful parameters which will come in handy later on
ntrain = train.shape[0]
ntest = test.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
kf = KFold(ntrain, n_folds= NFOLDS, random_state=SEED)

Create a class to extend the Sklearn classifier

In [33]:
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)
        

### Out-of-Fold Predictions
Stacking uses predictions of base classifiers as input for training to a second-level model. However one cannot simply train the base models on the full training data, generate predictions on the full test set and then output these for the second-level training. This runs the risk of your base model predictions already having "seen" the test set and therefore overfitting when feeding these predictions.

In [34]:
def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

## Generate base level models

### Random forest

In [35]:
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 500,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 0
}

In [36]:
# Create helper
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)

In [37]:
# Create OOF train and test predictions
rf_oof_train, rf_oof_test = get_oof(rf,x_train, y_train, x_test)

/home/treilly/Documents/projects/machine-learning-scratch/sklearn-titanic/venv/lib64/python3.6/site-packages/sklearn/ensemble/forest.py:305: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "


### Extra trees

In [38]:
# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators':500,
    #'max_features': 0.5,
    'max_depth': 8,
    'min_samples_leaf': 2,
    'verbose': 0
}

In [42]:
et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)

In [46]:
rf_oof_train, rf_oof_test = get_oof(rf,x_train, y_train, x_test)

/home/treilly/Documents/projects/machine-learning-scratch/sklearn-titanic/venv/lib64/python3.6/site-packages/sklearn/ensemble/forest.py:305: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "


### Adaboost

In [39]:
# AdaBoost parameters
ada_params = {
    'n_estimators': 500,
    'learning_rate' : 0.75
}

In [43]:
ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)

In [47]:
ada_oof_train, ada_oof_test = get_oof(ada, x_train, y_train, x_test)

### Graident boosting

In [40]:
# Gradient Boosting parameters
gb_params = {
    'n_estimators': 500,
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 0
}

In [44]:
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)

In [48]:
gb_oof_train, gb_oof_test = get_oof(gb,x_train, y_train, x_test)

### Support vector

In [41]:
# Support Vector Classifier parameters 
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025
}

In [45]:
svc = SklearnHelper(clf=SVC, seed=SEED, params=svc_params)

In [49]:
svc_oof_train, svc_oof_test = get_oof(svc,x_train, y_train, x_test)